#Обучение модели на датасете задания


Я использую модель SegFormer для семантической сегментации фотографий автомобилей. Модель я буду тонко настраивать на датасете из 211 фотографий. В семантической сегментации целью для модели является пометка каждого пикселя изображения одним из предопределенных классов.

Мы загружаем кодировщик модели с весами, предварительно обученными на ImageNet-1k, и тонко настраиваем его вместе с декодирующей головкой, которая начинается со случайно инициализированных весов.

In [ ]:
# Устанавливаем зависимости
!pip install -q transformers datasets

In [ ]:
# Подключаем google drive к файловой системе
from google.colab import drive
drive.mount('/content/drive')
aidir = '/content/drive/My Drive/ai/'

##Определяем пользовательский набор данных PyTorch.

 Каждый элемент набора данных состоит из изображения и соответствующей карты сегментации

In [ ]:
import os
from PIL import Image
import numpy as np

from torch.utils.data import Dataset


class SemanticSegmentationDataset(Dataset):
    def __init__(self, root_dir, feature_extractor, train=True):
        """
        Аргументы:
            root_dir (строка): Корневая папка датасета который состоит из картинок и масок
            feature_extractor (SegFormerFeatureExtractor): экстрактор признаков для подготовки картинок и масок
        """
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor

        self.img_dir = os.path.join(self.root_dir, "images")
        self.ann_dir = os.path.join(self.root_dir, "masks")

        # Считываем картинки
        image_file_names = []
        for root, dirs, files in os.walk(self.img_dir):
          image_file_names.extend(files)
        self.images = sorted(image_file_names)

        # Считываем маски
        annotation_file_names = []
        for root, dirs, files in os.walk(self.ann_dir):
          print(files)
          annotation_file_names.extend(files)
        self.annotations = sorted(annotation_file_names)

        assert len(self.images) == len(self.annotations), "Кол-во картинок не равно кол-ву масок"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.img_dir, self.images[idx]))
        image = image.convert("RGB")
        annotation = Image.open(os.path.join(self.ann_dir, self.annotations[idx]))

        # 2d маска на основе 3d
        annotation = np.array(annotation)
        annotation_2d = np.zeros((annotation.shape[0], annotation.shape[1]), dtype=np.uint8) # высота, ширина

        for id, color in id2color.items():
            annotation_2d[(annotation == color).all(axis=-1)] = id

        # случайное обрезание и заполнение изображения  и маски
        encoded_inputs = self.feature_extractor(image, Image.fromarray(annotation_2d), return_tensors="pt")

        for k,v in encoded_inputs.items():
          encoded_inputs[k].squeeze_() # удаляем размерность группы

        return encoded_inputs

In [ ]:
import pandas as pd

# Cчитываем цветовую палитру маски датасета
color_map = pd.read_csv(aidir + 'colors.txt',
                              sep=" ",
                              header=None)
color_map.columns = ["label_idx", "label", "R", "G", "B"]
color_map.head()

In [ ]:
label2id = {label: id for id, label in enumerate(color_map.label)}
id2label = {id: label for id, label in enumerate(color_map.label)}
id2color = {id: [R,G,B] for id, (R,G,B) in enumerate(zip(color_map.R, color_map.G, color_map.B))}

##Инициализация нашего датасета для обучения модели.





In [ ]:
from transformers import SegformerFeatureExtractor

root_dir = aidir + 'archive'
feature_extractor = SegformerFeatureExtractor()

train_dataset = SemanticSegmentationDataset(root_dir=root_dir, feature_extractor=feature_extractor)

In [ ]:
print("Кол-во примеров для обучения:", len(train_dataset))

Определяем загрузчик данных

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
batch = next(iter(train_dataloader))
mask = (batch["labels"] != 255)

## Настраиваем модель перед обучением

Загружаем модель Segformer с предобученными весами nvidia/mit-b0 (самая младшая модель). Задаём id2label и label2id которые нам понадобятся при инферансе (выводе) модели.

In [ ]:
from transformers import SegformerForSemanticSegmentation

model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b0",
                                                         num_labels=5,
                                                         id2label=id2label,
                                                         label2id=label2id,
)

## Тонкая настройка модели

Мы дополнительно обучаем нашу модель на датасете предоставленном к заданию с помощью оптимизатора AdamW. Для метрики обучения нашей модели используем IoU и точность на уровне пикселей.

In [ ]:
# Кол-во эпох
epoch_count = 200
# Через сколько эпох в цикле сохранять веса в папку (дефолт каждые 2 эпохи)
epoch_checkpoint = 2

In [ ]:
from datasets import load_metric

metric = load_metric("mean_iou")

In [ ]:
import os
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

# Оптимизатор
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
# Переносим модель на видеокарту
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Проверяем есть ли в нашей папке веса из предыдущих раундов обучения
weights_dir = aidir + 'weights'
if os.path.exists(weights_dir):
    # Если есть, то загружаем самые свежие веса
    weights_files = [f for f in os.listdir(weights_dir) if f.endswith('.pth')]
else:
    weights_files = []
if weights_files:
    weights_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
    latest_weights_file = weights_files[-1]
    print(f"Загружаем веса из файла: {latest_weights_file}")
    if torch.cuda.is_available():
      model.load_state_dict(torch.load(os.path.join(weights_dir, latest_weights_file)))
    else:
      model.load_state_dict(torch.load(os.path.join(weights_dir, latest_weights_file), map_location=torch.device('cpu')))
    start_epoch = int(latest_weights_file.split('_')[-1].split('.')[0]) + 1
else:
    os.makedirs(weights_dir, exist_ok=True)
    start_epoch = 0

model.train()
for epoch in range(start_epoch, epoch_count):  # Цикл прохода по датасету в течении нескольких эпох
   print("Epoch:", epoch)
   for idx, batch in enumerate(tqdm(train_dataloader)):
        # Получаем входные данные
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # Обнуляем параметры градиента
        optimizer.zero_grad()

        # Оптимизируем вперед и назад
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits

        loss.backward()
        optimizer.step()

        # Вычисляем веса
        with torch.no_grad():
          upsampled_logits = nn.functional.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
          predicted = upsampled_logits.argmax(dim=1)

          metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels.detach().cpu().numpy())

        # Будем выводить потери и метрики каждые 100 пакетов
        Выводим метрику модели
        if idx % 100 == 0:

          metrics = metric._compute(predictions=predicted.detach().cpu().numpy(),
                                   references=labels.detach().cpu().numpy(),
                                    num_labels=len(id2label),
                                   ignore_index=255,
                                   reduce_labels=False,
          )
          print("Loss:", loss.item())
          print("Mean_iou:", metrics["mean_iou"])

   if epoch % epoch_checkpoint == 0:
        weights_file = os.path.join(weights_dir, f'model_weights_{epoch}.pth')
        torch.save(model.state_dict(), weights_file)
        print(f'Сохранение весов модели в: {weights_file}')
